Training parameters: Working ion, crystal system, Spacegroup numbr
Output: max delta volume, volumetric capacity, gravimetric capacity

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [2]:
bat_data = pd.read_csv('TrainingData.csv')

In [3]:
bat_data

,Battery ID,Working Ion,Crystal System,Spacegroup Number,Gravimetric Capacity (units),Volumetric Capacity,Max Delta Volume,mean_Number,mean_MendeleevNumber,mean_AtomicWeight,...,dev_NValance.1,dev_NsUnfilled.1,dev_NpUnfilled.1,dev_NdUnfilled.1,dev_NfUnfilled.1,dev_NUnfilled.1,dev_GSvolume_pa.1,dev_GSbandgap.1,dev_GSmagmom.1,dev_SpaceGroupNumber.1
0,mp-504791_Li,Li,Orthorombic,19,50.228609,269.262771,0.028564,20.636364,80.090909,47.877178,...,4.055556,0.152778,0.791667,1.666667,0.00,1.166667,3.871508,0.248264,0.000000,81.791667
1,mp-763480_Li,Li,Monoclinic,14,39.674483,154.215698,0.017562,16.500000,82.333333,37.144083,...,2.725762,0.099723,0.581717,1.130194,0.00,0.880886,4.449100,0.432133,0.000000,58.204986
2,mp-1176966_Li,Li,Trigonal,165,181.943205,651.708695,0.061034,14.350000,83.500000,31.582317,...,1.918367,0.090703,0.464853,0.795918,0.00,0.727891,4.719358,0.501134,0.000000,45.969388
3,mvc-5592_Li,Li,Triclinic,2,73.484217,329.726946,0.021096,16.000000,82.600000,35.778332,...,2.429752,0.165289,0.661157,0.991736,0.00,0.842975,4.785895,0.483471,0.000000,65.223140
4,mp-763566_Li,Li,Monoclinic,4,73.484217,306.081406,0.025349,16.000000,82.600000,35.778332,...,2.429752,0.165289,0.661157,0.991736,0.00,0.842975,4.785895,0.483471,0.000000,65.223140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4396,mp-1180276_Mg,Mg,Cubic,215,92.144929,461.340369,0.069416,30.750000,74.250000,69.677437,...,3.950617,0.444444,0.987654,1.382716,0.00,1.111111,6.794568,0.394568,0.000000,101.530864
4397,mp-1042619_Mg,Mg,Cubic,204,60.789787,387.828816,0.006392,20.157895,81.894737,45.130042,...,3.195000,0.255000,0.800000,0.000000,0.00,0.740000,8.140500,0.000000,0.000000,80.220000
4398,mp-1046932_Mg,Mg,Triclinic,1,135.921407,914.229317,0.058047,24.100000,72.900000,55.509350,...,4.181474,0.000000,1.156900,0.574669,0.00,0.793951,11.118866,0.000000,0.171077,99.629490
4399,mp-1041103_Mg,Mg,Triclinic,1,112.348870,735.558063,0.019274,22.111111,72.777778,50.316302,...,1.340000,0.320000,0.960000,1.600000,0.54,1.380000,5.830000,0.000000,0.000000,95.760000


In [4]:
bat_data['Working Ion'].unique()

array(['Li', 'Ca', 'Cs', 'Rb', 'K', 'Y', 'Na', 'Al', 'Zn', 'Mg'],
      dtype=object)

1 Li
2 Ca
3 Cs
4 Rb
5 K
6 Y
7 Na
8 Al
9 Zn
0 Mg

In [5]:
bat_data['Crystal System'].unique()

array(['Orthorombic', 'Monoclinic', 'Trigonal', 'Triclinic', 'Tetragonal',
       'Hexagonal', 'Cubic'], dtype=object)

1 - Orthorombic
2 - Monoclinic
3 - Trigonal
4 - Triclinic
5 - Tetragonal
6 - Hexagonal
7 - Cubic


creating dataframe for training inputs:

In [6]:
train_bat = bat_data

In [7]:
train_bat = train_bat.replace('Li', 1)
train_bat = train_bat.replace('Ca', 2)
train_bat = train_bat.replace('Cs', 3)
train_bat = train_bat.replace('Rb', 4)
train_bat = train_bat.replace('K', 5)
train_bat = train_bat.replace('Y', 6)
train_bat = train_bat.replace('Na', 7)
train_bat = train_bat.replace('Al', 8)
train_bat = train_bat.replace('Zn', 9)
train_bat = train_bat.replace('Mg', 0)

In [8]:
train_bat

,Battery ID,Working Ion,Crystal System,Spacegroup Number,Gravimetric Capacity (units),Volumetric Capacity,Max Delta Volume,mean_Number,mean_MendeleevNumber,mean_AtomicWeight,...,dev_NValance.1,dev_NsUnfilled.1,dev_NpUnfilled.1,dev_NdUnfilled.1,dev_NfUnfilled.1,dev_NUnfilled.1,dev_GSvolume_pa.1,dev_GSbandgap.1,dev_GSmagmom.1,dev_SpaceGroupNumber.1
0,mp-504791_Li,1,Orthorombic,19,50.228609,269.262771,0.028564,20.636364,80.090909,47.877178,...,4.055556,0.152778,0.791667,1.666667,0.00,1.166667,3.871508,0.248264,0.000000,81.791667
1,mp-763480_Li,1,Monoclinic,14,39.674483,154.215698,0.017562,16.500000,82.333333,37.144083,...,2.725762,0.099723,0.581717,1.130194,0.00,0.880886,4.449100,0.432133,0.000000,58.204986
2,mp-1176966_Li,1,Trigonal,165,181.943205,651.708695,0.061034,14.350000,83.500000,31.582317,...,1.918367,0.090703,0.464853,0.795918,0.00,0.727891,4.719358,0.501134,0.000000,45.969388
3,mvc-5592_Li,1,Triclinic,2,73.484217,329.726946,0.021096,16.000000,82.600000,35.778332,...,2.429752,0.165289,0.661157,0.991736,0.00,0.842975,4.785895,0.483471,0.000000,65.223140
4,mp-763566_Li,1,Monoclinic,4,73.484217,306.081406,0.025349,16.000000,82.600000,35.778332,...,2.429752,0.165289,0.661157,0.991736,0.00,0.842975,4.785895,0.483471,0.000000,65.223140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4396,mp-1180276_Mg,0,Cubic,215,92.144929,461.340369,0.069416,30.750000,74.250000,69.677437,...,3.950617,0.444444,0.987654,1.382716,0.00,1.111111,6.794568,0.394568,0.000000,101.530864
4397,mp-1042619_Mg,0,Cubic,204,60.789787,387.828816,0.006392,20.157895,81.894737,45.130042,...,3.195000,0.255000,0.800000,0.000000,0.00,0.740000,8.140500,0.000000,0.000000,80.220000
4398,mp-1046932_Mg,0,Triclinic,1,135.921407,914.229317,0.058047,24.100000,72.900000,55.509350,...,4.181474,0.000000,1.156900,0.574669,0.00,0.793951,11.118866,0.000000,0.171077,99.629490
4399,mp-1041103_Mg,0,Triclinic,1,112.348870,735.558063,0.019274,22.111111,72.777778,50.316302,...,1.340000,0.320000,0.960000,1.600000,0.54,1.380000,5.830000,0.000000,0.000000,95.760000


In [9]:
train_bat = train_bat.replace('Orthorombic', 1)
train_bat = train_bat.replace('Monoclinic', 2)
train_bat = train_bat.replace('Trigonal', 3)
train_bat = train_bat.replace('Triclinic', 4)
train_bat = train_bat.replace('Tetragonal', 5)
train_bat = train_bat.replace('Hexagonal', 6)
train_bat = train_bat.replace('Cubic', 7)

In [10]:
train_bat

,Battery ID,Working Ion,Crystal System,Spacegroup Number,Gravimetric Capacity (units),Volumetric Capacity,Max Delta Volume,mean_Number,mean_MendeleevNumber,mean_AtomicWeight,...,dev_NValance.1,dev_NsUnfilled.1,dev_NpUnfilled.1,dev_NdUnfilled.1,dev_NfUnfilled.1,dev_NUnfilled.1,dev_GSvolume_pa.1,dev_GSbandgap.1,dev_GSmagmom.1,dev_SpaceGroupNumber.1
0,mp-504791_Li,1,1,19,50.228609,269.262771,0.028564,20.636364,80.090909,47.877178,...,4.055556,0.152778,0.791667,1.666667,0.00,1.166667,3.871508,0.248264,0.000000,81.791667
1,mp-763480_Li,1,2,14,39.674483,154.215698,0.017562,16.500000,82.333333,37.144083,...,2.725762,0.099723,0.581717,1.130194,0.00,0.880886,4.449100,0.432133,0.000000,58.204986
2,mp-1176966_Li,1,3,165,181.943205,651.708695,0.061034,14.350000,83.500000,31.582317,...,1.918367,0.090703,0.464853,0.795918,0.00,0.727891,4.719358,0.501134,0.000000,45.969388
3,mvc-5592_Li,1,4,2,73.484217,329.726946,0.021096,16.000000,82.600000,35.778332,...,2.429752,0.165289,0.661157,0.991736,0.00,0.842975,4.785895,0.483471,0.000000,65.223140
4,mp-763566_Li,1,2,4,73.484217,306.081406,0.025349,16.000000,82.600000,35.778332,...,2.429752,0.165289,0.661157,0.991736,0.00,0.842975,4.785895,0.483471,0.000000,65.223140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4396,mp-1180276_Mg,0,7,215,92.144929,461.340369,0.069416,30.750000,74.250000,69.677437,...,3.950617,0.444444,0.987654,1.382716,0.00,1.111111,6.794568,0.394568,0.000000,101.530864
4397,mp-1042619_Mg,0,7,204,60.789787,387.828816,0.006392,20.157895,81.894737,45.130042,...,3.195000,0.255000,0.800000,0.000000,0.00,0.740000,8.140500,0.000000,0.000000,80.220000
4398,mp-1046932_Mg,0,4,1,135.921407,914.229317,0.058047,24.100000,72.900000,55.509350,...,4.181474,0.000000,1.156900,0.574669,0.00,0.793951,11.118866,0.000000,0.171077,99.629490
4399,mp-1041103_Mg,0,4,1,112.348870,735.558063,0.019274,22.111111,72.777778,50.316302,...,1.340000,0.320000,0.960000,1.600000,0.54,1.380000,5.830000,0.000000,0.000000,95.760000


# Neural Network

In [147]:
dtype = torch.long
device = torch.device("cpu")

In [246]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 50, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-3
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 0.0005373313324525952
199 4.4525191866817693e-10
299 1.306197554631905e-12
399 1.0693002473055602e-12
499 1.0401126574605124e-12


In [54]:
loss

tensor(nan, grad_fn=<MseLossBackward>)

In [90]:
y_pred

tensor([[-0.1673,  0.6134, -1.1162,  0.1417,  0.4789,  1.9586,  0.3353,  0.5878,
          0.4940, -1.3649],
        [ 0.6992,  2.3176,  0.4821,  1.0397, -1.2691, -0.4594, -1.4908, -0.5396,
          1.0035, -0.9884],
        [ 0.9051,  1.7943, -1.0414, -0.7141, -0.5528, -0.3850, -2.2112,  1.3950,
          0.5643,  0.4708],
        [ 0.5781, -0.5711,  0.4450,  2.2548, -1.0628, -1.1913, -0.8369, -1.0021,
         -2.1210,  0.3702],
        [-0.8723,  0.8469, -1.1570, -0.6011, -0.0458,  1.9272,  0.6822,  0.8063,
          1.4981,  1.4312],
        [-1.3202,  0.9028,  1.1493, -0.0184, -1.0757, -1.1491,  1.2695, -1.4818,
         -0.8077, -0.0118],
        [ 0.1371,  1.3150,  0.9176,  0.3539, -1.5507,  0.8877, -0.0235, -1.0163,
          0.7782,  0.5028],
        [ 0.7388, -0.0629,  0.5925, -0.2773,  0.3519, -0.7115,  0.7633,  1.1638,
          0.2697, -2.1167],
        [ 0.6125,  0.9991,  1.2601, -0.7778, -0.6746, -0.3957, -0.8215, -0.4766,
         -1.2384, -0.7656],
        [-1.0392, -

In [23]:

from torch import nn

# define model architecture
model = nn.Sequential(
    nn.Linear(6, 4),
    nn.ReLU(),
    nn.Linear(4, 2),
    nn.ReLU(),
    nn.Linear(2, 1),
    nn.Sigmoid()
)

# print model architecture
print(model)

Sequential(
  (0): Linear(in_features=6, out_features=4, bias=True)
  (1): ReLU()
  (2): Linear(in_features=4, out_features=2, bias=True)
  (3): ReLU()
  (4): Linear(in_features=2, out_features=1, bias=True)
  (5): Sigmoid()
)


# Our Neural Network

Input:
1. Working Ion
2. Crystal System
3. Spacegroup

Output parameters:
1. Gravimetric Capacity
2. Max Delta Volume

In [11]:
model = nn.Sequential(
    nn.Linear(3, 10),
    nn.ReLU(),
    nn.Linear(10, 3),
    nn.Sigmoid()
)

In [12]:
loss_fn = torch.nn.MSELoss(reduction='sum')

In [18]:
D_in = 91
H = 9
D_out = 3
N = 4000
dtype = torch.float
device = torch.device('cpu')

#define testing sets
x_train = train_bat
x_train = x_train.drop(columns=['Battery ID', 'Gravimetric Capacity (units)', 'Volumetric Capacity', 'Max Delta Volume'])

y_train = train_bat[['Gravimetric Capacity (units)', 'Volumetric Capacity', 'Max Delta Volume']]

x_test = x_train[4000:]
y_test = y_train[4000:]

x_train = x_train[:4000]
y_train = y_train[:4000]

#Defining training and testing data
x_train_np = np.array(x_train)
x_train_torch = torch.tensor(x_train_np, device = device, dtype = dtype)

y_train_np = np.array(y_train)
y_train_torch = torch.tensor(y_train_np, device = device, dtype = dtype)

x_test_np = np.array(x_test)
x_test_torch = torch.tensor(x_test_np, device = device, dtype = dtype)

y_test_np = np.array(y_test)
y_test_torch = torch.tensor(y_test_np, device = device, dtype = dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

#define model
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

learning_rate = 1e-4
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x_train_torch)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y_train_torch)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 1330632192.0
199 1330632192.0
299 1330632192.0
399 1330632192.0
499 1330632192.0


In [19]:
D_in = 91
H = 100
D_out = 3
N = 4000
dtype = torch.float

#Defining training and testing data
x_train_np = np.array(x_train)
x_train_torch = torch.tensor(x_train_np, device = device, dtype = dtype)

y_train_np = np.array(y_train)
y_train_torch = torch.tensor(y_train_np, device = device, dtype = dtype)

x_test_np = np.array(x_test)
x_test_torch = torch.tensor(x_test_np, device = device, dtype = dtype)

y_test_np = np.array(y_test)
y_test_torch = torch.tensor(y_test_np, device = device, dtype = dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

#Implementing
learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x_train_torch.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # Compute and print loss
    loss = (y_pred - y_train_torch).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y_train_torch)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x_train_torch.t().mm(grad_h)

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 2871400960.0
199 2871400960.0
299 2871400960.0
399 2871400960.0
499 2871400960.0


In [20]:
import torch


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 3, 1000, 100, 10

# Create random input and output data
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 0.008589273318648338
199 4.057705780269316e-07
299 7.978171367994946e-08
399 4.116815333077284e-08
499 2.6121419693936332e-08


In [21]:
D_in = 91
H = 9
D_out = 3
N = 4000
dtype = torch.float

#define testing sets
x_train = train_bat
x_train = x_train.drop(columns=['Battery ID', 'Gravimetric Capacity (units)', 'Volumetric Capacity', 'Max Delta Volume'])

y_train = train_bat[['Gravimetric Capacity (units)', 'Volumetric Capacity', 'Max Delta Volume']]

x_test = x_train[4000:]
y_test = y_train[4000:]

x_train = x_train[:4000]
y_train = y_train[:4000]

#Defining training and testing data
x_train_np = np.array(x_train)
x_train_torch = torch.tensor(x_train_np, device = device, dtype = dtype)

y_train_np = np.array(y_train)
y_train_torch = torch.tensor(y_train_np, device = device, dtype = dtype)

x_test_np = np.array(x_test)
x_test_torch = torch.tensor(x_test_np, device = device, dtype = dtype)

y_test_np = np.array(y_test)
y_test_torch = torch.tensor(y_test_np, device = device, dtype = dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

#define model
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

learning_rate = 1e-4
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x_train_torch)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y_train_torch)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 1330632192.0
199 1330632192.0
299 1330632192.0
399 1330632192.0
499 1330632192.0


In [22]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 50, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-3
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 0.0010429106187075377
199 2.8422680875550554e-11
299 8.636295346399481e-13
399 8.619841494229841e-13
499 9.283919119584816e-13


In [259]:
import torch


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
D_in = 91
H = 9
D_out = 3
N = 4000
dtype = torch.float

#define testing sets
x_train = train_bat
x_train = x_train.drop(columns=['Battery ID', 'Gravimetric Capacity (units)', 'Volumetric Capacity', 'Max Delta Volume'])

y_train = train_bat[['Gravimetric Capacity (units)', 'Volumetric Capacity', 'Max Delta Volume']]

x_test = x_train[4000:]
y_test = y_train[4000:]

x_train = x_train[:4000]
y_train = y_train[:4000]

#Defining training and testing data
x_train_np = np.array(x_train)
x_train_torch = torch.tensor(x_train_np, device = device, dtype = dtype)

y_train_np = np.array(y_train)
y_train_torch = torch.tensor(y_train_np, device = device, dtype = dtype)

x_test_np = np.array(x_test)
x_test_torch = torch.tensor(x_test_np, device = device, dtype = dtype)

y_test_np = np.array(y_test)
y_test_torch = torch.tensor(y_test_np, device = device, dtype = dtype)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x_train_torch.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # Compute and print loss
    loss = (y_pred - y_train_torch).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y_train_torch)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x_train_torch.t().mm(grad_h)

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 2871400960.0
199 2871400960.0
299 2871400960.0
399 2871400960.0
499 2871400960.0
